# Run Structures Stored in Neo4j

We support the direct execution of structures stored in Neo4j.

For more information on constructing structures, please refer to the following notebooks:

- lion_agent_basic: Building basic structure
- lion_agent_conditional_path: Adding conditions on edges for conditional execution
- lion_agent_multi_path: Running multiple paths in parallel
- lion_agent_nested_agents: Executing other agents as within the workflow
- lion_agent_tool_action_selection: Enabling function calling and modifying execution mode

## Initiate Neo4j Driver

In [1]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "12345678"
database = "neo4j"

from lionagi.integrations.storage.neo4j import Neo4j

neo4j = Neo4j(uri, user, password, database)

In [2]:
from lionagi.core.agent.base_agent import BaseAgent
from lionagi.core.execute.instruction_map_executor import InstructionMapExecutor
from lionagi.core.execute.neo4j_executor import Neo4jExecutor

In [3]:
def output_parser(agent):
    output = []
    for branch in agent.executable.branches.values():
        output.append(branch.messages)
    return output

### Reload lion_agent_conditional_path

In [4]:
structure = Neo4jExecutor(driver=neo4j, structure_name="math")
executor = InstructionMapExecutor()
calculator1 = BaseAgent(
    structure=structure, executable=executor, output_parser=output_parser
)

In [5]:
result_calc = await calculator1.execute(context={"x": -6, "y": 0, "case": 0})

------------------Welcome: system--------------------


system: You are asked to perform as a calculator. Return only a numeric value, i.e. int or float, no text.

user: {"sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|"}

assistant: 6

-----------------------------------------------------


user: {"if previous response is positive": "times 2. i.e. *2", "else": "plus 2. i.e. +2"}

assistant: 12

-----------------------------------------------------


### Reload lion_agent_multi_path

In [6]:
structure = Neo4jExecutor(driver=neo4j, structure_name="math_multi")
executor = InstructionMapExecutor()
calculator2 = BaseAgent(
    structure=structure, executable=executor, output_parser=output_parser
)

In [7]:
result_calc2 = await calculator2.execute(context={"x": 2, "y": 3})

------------------Welcome: system--------------------


system: You are asked to perform as a calculator. Return only a numeric value, i.e. int or float, no text.

user: {"diff the absolute values": "provided with 2 numbers, return the difference of absolute values. i.e. |x|-|y|"}

user: {"sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|"}

assistant: 5

...

user: {"if previous response is positive": "times 2. i.e. *2", "else": "plus 2. i.e. +2"}

assistant: 1

-----------------------------------------------------


assistant: 10

-----------------------------------------------------


### Reload lion_agent_nested_agents

In [8]:
structure = Neo4jExecutor(driver=neo4j, structure_name="comedian")
executor = InstructionMapExecutor()
comedian = BaseAgent(
    structure=structure, executable=executor, output_parser=output_parser
)

In [9]:
comedian_result = await comedian.execute()

------------------Welcome: comedian--------------------


system: As a comedian, you are sarcastically funny

user: very short joke: a blue whale and a big shark meet at the bar and start dancing

assistant: Why did the blue whale and the big shark start dancing at the bar? Because they heard the bartender say, "Let's make this place a bit more fintastic!"

-----------------------------------------------------


...

assistant: Why did the blue whale and the big shark hit the dance floor at the bar? Because the bartender said, 'Let's shake up some waves and make this spot fintastically lively!'

-----------------------------------------------------


user: write joke2 again

assistant: They abruptly stopped grooving when 'Baby Shark' came on. The shark groaned, 'Every time! That's my ex's favorite jam. Talk about a mood killer!'

-----------------------------------------------------


### Reload lion_agent_tool_action_selection

In [10]:
structure = Neo4jExecutor(driver=neo4j, structure_name="tool_mul")
executor = InstructionMapExecutor()
calc_mul = BaseAgent(
    structure=structure, executable=executor, output_parser=output_parser
)

In [11]:
question1 = "A school is ordering laptops for its students. If each classroom has 25 students and the school wants to provide a laptop for each student in its 8 classrooms, how many laptops in total does the school need to order?"
question2 = "A bakery sells cupcakes in boxes of 6. If a customer wants to buy enough cupcakes for a party of 48 people, with each person getting one cupcake, how many boxes of cupcakes does the customer need to buy?"

import json

context = {"Question1": question1, "question2": question2}
context = json.dumps(context)

In [12]:
calc_result = await calc_mul.execute(context)

------------------Welcome: system--------------------


system: you are asked to perform as a function picker and parameter provider

user: Think step by step, understand the following basic math question and provide parameters for function calling.

assistant: The final results for the given math questions are as follows:

1. **For the first question**: The school needs to order a total of **200 laptops** for its students. This is computed by multiplying the number of students per classroom (25) by the number of classrooms (8).

2. **For the second question**: Although there was a misinterpretation in calculating the number of boxes of cupcakes needed by directly applying multiplication rather than division and rounding up, the intended calculation should determine how many boxes are needed for 48 cupcakes, given that each box contains 6 cupcakes. The correct operation would involve dividing 48 (the total number of cupcakes needed) by 6 (the number of cupcakes per box) and rounding up if necessary. However, since 48 is a multiple of 6, **8 boxes** are exactly needed without the need for rounding up, which aligns with the output mistakenly requested (48 cupcakes require 8 boxes at 6 cupcakes per box).

Apologies for the confusion in the process. The essential takeaway is that the school needs to order 200 laptops, and for the party, 8 boxes of cupcakes are needed to ensure each of the 48 attendees receives one cupcake.

-----------------------------------------------------
